In [2]:
import xarray as xr
import pandas as pd
import numpy as np

In [3]:
def power_curve(speed):
    if speed < 3 or speed > 25:
        cf = 0
    elif 3 <= speed <= 13:
        cf = 1/2*1.225*50**2*np.pi*speed**3 / 10600000 - 0.012
    elif 13 < speed <= 25:
        cf = 1
    return cf

def onshore_cf(u, v):
    s = np.sqrt(u["u"]**2 + v["v"]**2).to_dataset(name="speed")
    s.attrs["units"] = "m/s"
    cf = xr.apply_ufunc(np.interp, s, np.arange(0, 100.01, 0.1), np.array(list(map(power_curve, np.arange(0, 100.01, 0.1)))))
    return cf

In [6]:
year = "2022"
# Onshore wind turbine generation potential
print(f"Processing {year}")
# Load wind speed data
u = xr.open_dataset(f"/Volumes/Results/ERA5/era5.u_component_of_wind/era5.u_component_of_wind.{year}.nc")
v = xr.open_dataset(f"/Volumes/Results/ERA5/era5.v_component_of_wind/era5.v_component_of_wind.{year}.nc")
cf = onshore_cf(u, v)
cf.to_netcdf(f"/Volumes/Results/ERA5-CF/era5.onshore_cf/era5.onshore_cf.{year}.nc")

Processing 2022
